# Libraries

In [7]:
import numpy  as np
import pandas as pd
import time

from pkg.constants    import *
from pkg.evaluation   import Evaluation
from pkg.utils        import *
from pkg.algorithims  import *

# VNS

## Parámetros del VNS

In [2]:
# semillas usadas para la experimentacion
seeds = np.array([7054, 1354, 23503, 11268, 58283])

# Número máximo de entornos explorados
k_max = 4

# Tamaños de los entornos (en orden creciente)
sizes_of_sublists = [4, 8, 12, 16]

# Granularidad del movimiento, calculada previamente
granularity = 2 

# Numero de intentos (mutaciones fuertes) al llegar al ultimo entorno
attempts = 5

In [4]:
ev = Evaluation()

In [6]:
# Calculamos el tiempo medio de ejecución de 1 evaluación

t_total = 0

for i in range(100):
    rs = generate_random_solution()

    ini = time.time()
    ev.evaluate(rs)
    fin = time.time()

    t_total += (fin-ini)

t_medio = t_total/100
print(t_medio) 

0.05094067573547363


Calculamos el parámetro **bl_max**, que indica el criterio de salida. Para calcularlo se tienen en cuenta los siguientes datos:
- Número de evaluaciones medias por BL: 892.6 evaluaciones (dato obtenido de la práctica anterior)
- Tiempo de ejecución de 1 evaluación: 0.05094067573547363 segundos

Por tanto, el tiempo de ejecución medio de una BL es de 45 segundos aproximadamente.

El valor de **bl_max** se establece de tal forma que el tiempo de ejecución de una búsqueda VNS no supere los 30 minutos. Como una BL tarda de media menos de 1 minuto, podemos establecer que bl_max = 30. De esta forma, aunque nos encontremos con una BL con una duración mayor a la media, el tiempo aproximado de la búsqueda VNS seguirá rondando los 30 minutos.

In [5]:
bl_max = 30

## Operador de generador de vecino

Comprobamos el correcto funcionamiento del operador de generador de vecino

In [6]:
ks = list(range(1,k_max+1))

for k in ks:

    solution = generate_random_solution()

    neighbor = neighbor_generation_operator(
        solution    = solution, 
        k           = k,
        sizes_Ek    = sizes_of_sublists,
        granularity = granularity
    )

    print(f'\nk = {k}')
    print(solution)
    print(neighbor)


k = 1
[ 6 17  9 17 25  6  6 14  6 17 11 14 26  9 11 26]
[ 6 17  9 17 25  6  6 14  6 17 11 14 24 11  9 28]

k = 2
[15 24 15 12  9  6 28  9 15 15  6 12  9 12 21  9]
[15 24 15 12  9  6 28  9 13 17  4 14  7 14 19 11]

k = 3
[ 5 11 19  5 19 11 19 16  8 14 16  8 19 11 19 19]
[ 7 11 19  5 19  9 21 14 10 12 18  6 21  9 21 17]

k = 4
[10 19 14 14 19 21 14  5 10 22 17 10 14 19  5  7]
[12 17 16 12 21 19 16  3 12 20 19  8 16 17  7  5]


## Experimentación Algoritmo

In [9]:
costs       = np.array([])
evaluations = np.array([])
slots       = np.array([])

for s in seeds:

    # set evauation calls counter to 0
    ev.total_calls = 0

    np.random.seed(s)

    solution = generate_random_solution()
    sol, cost = vns(
        solution    = solution,
        k_max       = k_max,
        bl_max      = bl_max,
        sizes_Ek    = sizes_of_sublists,
        granularity = granularity,
        ev          =ev
    )

    print(solution)
    print(cost)
    
    costs       = np.append(costs, cost)
    evaluations = np.append(evaluations, ev.total_calls)
    slots       = np.append(slots, solution.sum())

df = pd.DataFrame({
    "ev_medias"    : [evaluations.mean()],
    "ev_mejor"     : [evaluations.min()],
    "ev_std"       : [evaluations.std()],
    "coste_medio"  : [costs.mean()],
    "coste_mejor"  : [costs.min()],
    "coste_std"    : [costs.std()],
    "slots_medio"  : [slots.mean()],
    "slots_min"    : [slots.min()],
    "slots_std"    : [slots.std()]
})

df_2 = pd.DataFrame({
    'Ejecución'    : [1,2,3,4,5],
    "Coste mejor"  : costs,
    "Evaluaciones" : evaluations,
    "slots"        : slots
})
    

1
mejora
Coste antiguo: 564.9951766145007
Coste nuevo: 375.7658566586277
2
no mejora
3
no mejora
4
no mejora
5
no mejora
[22 10 17 15 12  5 20  7 10 15 10 10 12 20 20 15]
375.7658566586277
1
mejora
Coste antiguo: 522.6903616556751
Coste nuevo: 379.4289296151759
2
no mejora
3
mejora
Coste antiguo: 379.4289296151759
Coste nuevo: 377.0153450666003
4
no mejora
5
no mejora
6
no mejora
7
no mejora
[14 10 14 12 14 21 22 19  5  7 14 10 12 10 14 22]
377.0153450666003
1
mejora
Coste antiguo: 570.9421348371571
Coste nuevo: 393.843538106775
2
no mejora
3
no mejora
4
no mejora
5
no mejora
[ 8  8 24  8 16 14  5 22 14 16  5 16 16 16 19 11]
393.843538106775
1
mejora
Coste antiguo: 508.07413744520755
Coste nuevo: 377.2167786072197
2
no mejora
3
mejora
Coste antiguo: 377.2167786072197
Coste nuevo: 374.6904512366752
4
no mejora
5
no mejora
6
no mejora
7
mejora
Coste antiguo: 374.6904512366752
Coste nuevo: 371.85391617579
8
no mejora
9
no mejora
10
no mejora
11
no mejora
[ 9  6  6 23 15 12 12 18 21 12 15 

In [10]:
df

,ev_medias,ev_mejor,ev_std,coste_medio,coste_mejor,coste_std,slots_medio,slots_min,slots_std
0,3845.8,2201.0,1381.946656,385.630852,371.853916,14.191177,218.8,216.0,1.6


In [11]:
df_2

,Ejecución,Coste mejor,Evaluaciones,slots
0,1,375.765857,2496.0,220.0
1,2,377.015345,4261.0,220.0
2,3,393.843538,2201.0,218.0
3,4,371.853916,6008.0,220.0
4,5,409.675606,4263.0,216.0


In [13]:
df.to_excel('./01_stats_p2/vns_stats.xlsx')
df_2.to_excel('./01_stats_p2/vns_ejecuciones.xlsx')

## Experimentación Algoritmo mejorado

Debido a que el algoritmo se estanca antes de llegar al número máximo de búsquedas locales (bl_max = 30), incorporamos una mejora del algoritmo. De esta forma, en una última instancia, se incorpora el parámetro "intentos", que realiza 5 mutaciones fuertes (fuertes porque el entorno es k=4), dado que cada mutación arroja un resultado distinto.

In [14]:
costs       = np.array([])
evaluations = np.array([])
slots       = np.array([])

for s in seeds:

    # set evauation calls counter to 0
    ev.total_calls = 0
    
    np.random.seed(s)

    solution = generate_random_solution()
    sol, cost = vns_upgrade(
        solution = solution,
        k_max = k_max,
        bl_max = bl_max,
        attempts= attempts,
        sizes_Ek= sizes_of_sublists,
        granularity=granularity,
        ev=ev
    )

    print(solution)
    print(cost)
    
    costs       = np.append(costs, cost)
    evaluations = np.append(evaluations, ev.total_calls)
    slots       = np.append(slots, solution.sum())


df = pd.DataFrame({
    "ev_medias"    : [evaluations.mean()],
    "ev_mejor"     : [evaluations.min()],
    "ev_std"       : [evaluations.std()],
    "coste_medio"  : [costs.mean()],
    "coste_mejor"  : [costs.min()],
    "coste_std"    : [costs.std()],
    "slots_medio"  : [slots.mean()],
    "slots_min"    : [slots.min()],
    "slots_std"    : [slots.std()]
})

df_2 = pd.DataFrame({
    'Ejecución'    : [1,2,3,4,5],
    "Coste mejor"  : costs,
    "Evaluaciones" : evaluations,
    "slots"        : slots
})

1
mejora
Coste antiguo: 564.9951766145007
Coste nuevo: 375.7658566586277
2
no mejora
3
no mejora
4
no mejora
5
no mejora
6
no mejora
7
no mejora
8
no mejora
9
no mejora
[22 10 17 15 12  5 20  7 10 15 10 10 12 20 20 15]
375.7658566586277
1
mejora
Coste antiguo: 522.6903616556751
Coste nuevo: 379.4289296151759
2
no mejora
3
mejora
Coste antiguo: 379.4289296151759
Coste nuevo: 377.0153450666003
4
no mejora
5
no mejora
6
no mejora
7
no mejora
8
no mejora
9
mejora
Coste antiguo: 377.0153450666003
Coste nuevo: 376.0421355251909
10
no mejora
11
no mejora
12
no mejora
13
no mejora
14
no mejora
15
no mejora
16
no mejora
17
no mejora
[14 10 14 12 14 21 22 19  5  7 14 10 12 10 14 22]
376.0421355251909
1
mejora
Coste antiguo: 570.9421348371571
Coste nuevo: 393.843538106775
2
no mejora
3
no mejora
4
no mejora
5
no mejora
6
no mejora
7
no mejora
8
no mejora
9
no mejora
[ 8  8 24  8 16 14  5 22 14 16  5 16 16 16 19 11]
393.843538106775
1
mejora
Coste antiguo: 508.07413744520755
Coste nuevo: 377.21677

In [15]:
df

,ev_medias,ev_mejor,ev_std,coste_medio,coste_mejor,coste_std,slots_medio,slots_min,slots_std
0,6780.6,4006.0,2258.586779,385.43621,371.853916,14.314151,218.8,216.0,1.6


In [16]:
df_2

,Ejecución,Coste mejor,Evaluaciones,slots
0,1,375.765857,4707.0,220.0
1,2,376.042136,9952.0,220.0
2,3,393.843538,4006.0,218.0
3,4,371.853916,8630.0,220.0
4,5,409.675606,6608.0,216.0


In [17]:
df.to_excel('./01_stats_p2/vns_upgrade_stats.xlsx')
df_2.to_excel('./01_stats_p2/vns_upgrade_ejecuciones.xlsx')

No se ve mejoria ninguna en los resultados.

Posibles enfoques: aumentar el numero de intentos en el ultimo entorno.
                    Probar con nuevas semillas, igual las soluciones iniciales no son buenas.